In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import IPython.display as ipd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import torch
import h5py
import logging
import sys

In [4]:
from parallel_wavegan.models.asr import ConvASREncoder
from parallel_wavegan.models.asr import MaskedConv1d

In [5]:
from omegaconf import DictConfig, ListConfig, OmegaConf
from collections import OrderedDict

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
model_state_file="/opt/storage/kurofune/nemo/QuartzNet15x5Base-En.pt"

In [7]:
conf_file="/workspace/parallel_wavegan/encoder-conf/quartznet_15x5.yaml"

In [8]:
conf = OmegaConf.load(conf_file)

In [9]:
encoder_conf = conf.model.encoder
jasper = encoder_conf.jasper
feat_in = encoder_conf.feat_in
activation=encoder_conf.activation
conv_mask = encoder_conf.conv_mask

In [10]:
encoder= ConvASREncoder(jasper, activation, feat_in, conv_mask=True)

In [11]:
state_dict = torch.load(model_state_file)

In [12]:
encoder_state_dict = OrderedDict()

In [13]:
for k, v in encoder.state_dict().items():
    if k.startswith("encoder"):
        print(k)
        break

encoder.0.mconv.0.conv.weight


In [14]:
for k, v in state_dict.items():
    new_key = k.replace("encoder.encoder", "encoder")
    if k.startswith("encoder"):
        encoder_state_dict[new_key] = v

In [15]:
list(encoder_state_dict.keys())[0:5]

['encoder.0.mconv.0.conv.weight',
 'encoder.0.mconv.1.conv.weight',
 'encoder.0.mconv.2.weight',
 'encoder.0.mconv.2.bias',
 'encoder.0.mconv.2.running_mean']

In [16]:
encoder.load_state_dict(encoder_state_dict)

<All keys matched successfully>

In [17]:
_ = encoder.eval()

In [18]:
_ = encoder.to(device)

In [26]:
for m in encoder.modules():
    if isinstance(m, MaskedConv1d):
        m.use_mask = False

In [9]:
file="/opt/storage/kurofune/parallel-wave-gan/data/lj-16000/train_nodev/norm/dump.1/LJ001-0001.h5"
with h5py.File(file, "r") as f:
    print(f.keys())
    nemofeats = f["nemofeats"][:]
    nn=f["nemofeats-5x5-4th"][:]
    wav = f["wave"][:]
    feats=f["feats"][:]
c =  torch.from_numpy(feats.reshape(1, -1, 64)).permute(0,2,1)

<KeysViewHDF5 ['feats', 'nemofeats', 'nemofeats-5x5-4th', 'wave']>


In [11]:
nn.shape,nemofeats.shape

((483, 512), (483, 512))

In [47]:
x = c.to(device)
length = torch.tensor([x.shape[2]]).to(device)
y_, _ = encoder.encoder[0:4](([x],None))

In [33]:
y_[0].shape

torch.Size([1, 256, 483])

In [41]:
x.shape

torch.Size([1, 64, 966])